In [2]:
import numpy as np
import pandas as pd

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [6]:
hr = pd.read_csv('D:/Datasets/HRAnalytics.csv')

In [4]:
hr.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [15]:
hr_dummies = pd.get_dummies(hr)
train, test = train_test_split(hr_dummies, test_size = 0.3, random_state = 100)
train_y = train['Attrition']
test_y = test['Attrition']

train_x = train.drop('Attrition', axis = 1)
test_x = test.drop('Attrition', axis = 1)

from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=100)
model_rf.fit(train_x, train_y)

test_pred = model_rf.predict(test_x)
df_pred = pd.DataFrame({'actual' : test_y,
                       'predicted' : test_pred})
df_pred['pred_status'] = df_pred['actual'] == df_pred['predicted']
df_pred['pred_status'].sum() / df_pred.shape[0] * 100

85.71428571428571

In [18]:
df_pred.head()

,actual,predicted,pred_status
880,0,0,True
152,0,0,True
1466,0,0,True
1084,0,0,True
1086,0,0,True


In [16]:
## Build a confusion Matrix and find the number of TP,TN,FP,FN

from sklearn.metrics import confusion_matrix
confusion_matrix(df_pred['actual'],df_pred['predicted'])

array([[367,   4],
       [ 59,  11]], dtype=int64)

In [17]:
#True Positive
tp = df_pred[(df_pred['predicted']==1) & (df_pred['actual']==1)].shape[0]
# True Negative
tn = df_pred[(df_pred['predicted']==0) & (df_pred['actual']==0)].shape[0]
#False Postive
fp = df_pred[(df_pred['predicted']==1) & (df_pred['actual']==0)].shape[0]
#False Negative
fn = df_pred[(df_pred['predicted']==0) & (df_pred['actual']==1)].shape[0]

print('True Positive %d' % tp)
print('True Negative %d' % tn)
print('False Positive %d' % fp)
print('False Negative %d' % fn)

True Positive 11
True Negative 367
False Positive 4
False Negative 59


In [20]:
# Confusion Matrix function from sklearn package
tn,fp,fn,tp = confusion_matrix(df_pred['actual'],df_pred['predicted']).ravel()
print(tn,fp,fn,tp)

367 4 59 11


In [21]:
# report in sklearn
from sklearn.metrics import classification_report
print(classification_report(df_pred['actual'],df_pred['predicted']))

             precision    recall  f1-score   support

          0       0.86      0.99      0.92       371
          1       0.73      0.16      0.26        70

avg / total       0.84      0.86      0.82       441



In [ ]:
# Banking Dataset , build a decision tree,Random Forest and adaptive boost model 

### ADAPTIVE BOOST ( AdaBoost)

In [22]:
from sklearn.ensemble import AdaBoostClassifier

In [27]:
model  = AdaBoostClassifier(random_state=100)
model.fit(train_x,train_y)
test_pred = model.predict(test_x)
tn,fp,fn,tp = confusion_matrix(test_y,test_pred).ravel() # ravel is for flattening
tn,fp,fn,tp

(352, 19, 43, 27)

In [28]:
acc = (tp + tn) / (tp + tn + fp + fn)
acc

0.8594104308390023

In [29]:
sensitivity = tp / (tp + fn)
sensitivity

0.38571428571428573

### BANKING DATASET : Decision Tree , Random Forest and AdaptiveBoosting Models

In [7]:
bank = pd.read_csv('D:/Datasets/bank-full.csv',sep = ';')

In [33]:
bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


## Decision Tree

In [19]:
bank_new = bank.drop('y', axis = 1)
bank_dummies = pd.get_dummies(bank_new)

bank_dummies['y'] = bank['y']
print(bank_dummies.shape)

train, test = train_test_split(bank_dummies, test_size = 0.3, random_state = 100)

train_y = train['y']
test_y = test['y']
train_x = train.drop('y', axis = 1)
test_x = test.drop('y', axis = 1)

model = DecisionTreeClassifier(random_state=100)
model.fit(train_x, train_y)

test_pred = model.predict(test_x)
df_pred = pd.DataFrame({'actual' : test_y,
                       'predicted' : test_pred})
df_pred['pred_status'] = df_pred['actual'] == df_pred['predicted']
acc = df_pred['pred_status'].sum() / df_pred.shape[0] * 100
acc

(45211, 52)


87.72485992332645

### Parameters

In [20]:
#True Positive
tp_dt = df_pred[(df_pred['predicted']=='yes') & (df_pred['actual']=='yes')].shape[0]
# True Negative
tn_dt = df_pred[(df_pred['predicted']=='no') & (df_pred['actual']=='no')].shape[0]
#False Postive
fp_dt = df_pred[(df_pred['predicted']=='yes') & (df_pred['actual']=='no')].shape[0]
#False Negative
fn_dt = df_pred[(df_pred['predicted']=='no') & (df_pred['actual']=='yes')].shape[0]

print('True Positive %d' % tp_dt)
print('True Negative %d' % tn_dt)
print('False Positive %d' % fp_dt)
print('False Negative %d' % fn_dt)

acc_dt = (tp_dt + tn_dt) / (tp_dt + tn_dt + fp_dt + fn_dt)
print('Accuracy',acc_dt)

sensitivity_dt = tp_dt / (tp_dt + fn_dt)
print('Sensitivity',sensitivity_dt)

spec_dt = tn_dt/(fp_dt + tn_dt)
print('Specificity',spec_dt)

True Positive 775
True Negative 11124
False Positive 861
False Negative 804
Accuracy 0.8772485992332645
Sensitivity 0.4908169727675744
Specificity 0.9281602002503129


## Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier
model_rf = RandomForestClassifier(random_state=100)
model_rf.fit(train_x, train_y)

test_pred = model_rf.predict(test_x)
df_pred1 = pd.DataFrame({'actual' : test_y,
                       'predicted' : test_pred})
df_pred1['pred_status'] = df_pred1['actual'] == df_pred1['predicted']
df_pred1['pred_status'].sum() / df_pred1.shape[0] * 100

89.97345915659098

### Parameters

In [22]:
#True Positive
tp_rf = df_pred1[(df_pred1['predicted']=='yes') & (df_pred1['actual']=='yes')].shape[0]
# True Negative
tn_rf = df_pred1[(df_pred1['predicted']=='no') & (df_pred1['actual']=='no')].shape[0]
#False Postive
fp_rf = df_pred1[(df_pred1['predicted']=='yes') & (df_pred1['actual']=='no')].shape[0]
#False Negative
fn_rf = df_pred1[(df_pred1['predicted']=='no') & (df_pred1['actual']=='yes')].shape[0]

print('True Positive %d' % tp_rf)
print('True Negative %d' % tn_rf)
print('False Positive %d' % fp_rf)
print('False Negative %d' % fn_rf)

acc_rf = (tp_rf + tn_rf) / (tp_rf + tn_rf + fp_rf + fn_rf)
print('Accuracy',acc_rf)

sensitivity_rf = tp_rf / (tp_rf + fn_rf)
print('Sensitivity',sensitivity_rf)

spec_rf = tn_rf/(fp_rf + tn_rf)
print('Specificity',spec_rf)

True Positive 561
True Negative 11643
False Positive 342
False Negative 1018
Accuracy 0.8997345915659097
Sensitivity 0.3552881570614313
Specificity 0.9714643304130163


## AdaBoost

In [23]:
from sklearn.ensemble import AdaBoostClassifier
model_ab  = AdaBoostClassifier(random_state=100)
model_ab.fit(train_x,train_y)
test_pred = model_ab.predict(test_x)
df_pred2 = pd.DataFrame({'actual' : test_y,
                       'predicted' : test_pred})
df_pred2['pred_status'] = df_pred2['actual'] == df_pred2['predicted']
df_pred2['pred_status'].sum() / df_pred2.shape[0] * 100
#tn,fp,fn,tp = confusion_matrix(test_y,test_pred).ravel() # ravel is for flattening
#tn,fp,fn,tp

89.72279563550575

### Parameters

In [24]:
#True Positive
tp_ab = df_pred2[(df_pred2['predicted']=='yes') & (df_pred2['actual']=='yes')].shape[0]
# True Negative
tn_ab = df_pred2[(df_pred2['predicted']=='no') & (df_pred2['actual']=='no')].shape[0]
#False Postive
fp_ab = df_pred2[(df_pred2['predicted']=='yes') & (df_pred2['actual']=='no')].shape[0]
#False Negative
fn_ab = df_pred2[(df_pred1['predicted']=='no') & (df_pred2['actual']=='yes')].shape[0]

print('True Positive %d' % tp_ab)
print('True Negative %d' % tn_ab)
print('False Positive %d' % fp_ab)
print('False Negative %d' % fn_ab)

acc_ab = (tp_ab + tn_ab) / (tp_ab + tn_ab + fp_ab + fn_ab)
print('Accuracy',acc_ab)

sensitivity_ab = tp_ab / (tp_ab + fn_ab)
print('Sensitivity',sensitivity_ab)

spec_ab = tn_ab/(fp_ab + tn_ab)
print('Specificity',spec_ab)

True Positive 585
True Negative 11585
False Positive 400
False Negative 1018
Accuracy 0.8956432146011186
Sensitivity 0.3649407361197754
Specificity 0.966624947851481


# KNN

In [14]:
#Knn

from sklearn.neighbors import KNeighborsClassifier

In [8]:
bank_new = bank.drop('y', axis = 1)
bank_dummies = pd.get_dummies(bank_new)

bank_dummies['y'] = bank['y']
print(bank_dummies.shape)

train, test = train_test_split(bank_dummies, test_size = 0.3, random_state = 100)

train_y = train['y']
test_y = test['y']
train_x = train.drop('y', axis = 1)
test_x = test.drop('y', axis = 1)

(45211, 52)


In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
bank_scaled = scaler.fit_transform(bank_dummies.drop('y', axis=1))
bank_scaled = pd.DataFrame(bank_scaled, 
                         columns=bank_dummies.drop('y', axis=1).columns)
bank_scaled['y'] = bank['y']
#hr_scaled.head(1)

In [17]:
model_knn = KNeighborsClassifier(n_neighbors=5)

model_knn.fit(train_x, train_y)

test_pred_knn = model_knn.predict(test_x)

df_pred_knn = pd.DataFrame({'actual': test_y,
                        'predicted': test_pred_knn})

df_pred_knn['pred_status'] = df_pred_knn['actual'] == df_pred_knn['predicted']

## true positive
tp_knn = df_pred_knn[(df_pred_knn['predicted']=='yes') & (df_pred_knn['actual']=='yes')].shape[0]

###true negative
tn_knn = df_pred_knn[(df_pred_knn['predicted']=='no') & (df_pred_knn['actual']=='no')].shape[0]

##false positive
fp_knn = df_pred_knn[(df_pred_knn['predicted']=='yes') & (df_pred_knn['actual']=='no')].shape[0]

##false negative
fn_knn = df_pred_knn[(df_pred_knn['predicted']=='no') & (df_pred_knn['actual']=='yes')].shape[0]

print('True Positive %d' % tp_knn)
print('True Negative %d' % tn_knn)
print('False Positive %d' % fp_knn)
print('False Negative %d' % fn_knn)

acc_knn = (tp_knn + tn_knn) / (tp_knn + tn_knn + fp_knn + fn_knn)
print('Accuracy',acc_knn)

sensitivity_knn = tp_knn / (tp_knn + fn_knn)
print('Sensitivity',sensitivity_knn)

spec_knn = tn_knn/(fp_knn + tn_knn)
print('Specificity',spec_knn)

True Positive 434
True Negative 11510
False Positive 475
False Negative 1145
Accuracy 0.8805662046593925
Sensitivity 0.27485750474984166
Specificity 0.9603671255736337


## SUMMARY

In [25]:
dt =pd.Series({'TP':tp_dt,'TN':tn_dt,'FP':fp_dt,'FN':fn_dt,'Accuracy':acc_dt,'Sensitivity':sensitivity_dt,'Specificity':spec_dt})
rf =pd.Series({'TP':tp_rf,'TN':tn_rf,'FP':fp_rf,'FN':fn_rf,'Accuracy':acc_rf,'Sensitivity':sensitivity_rf,'Specificity':spec_rf})
ab =pd.Series({'TP':tp_ab,'TN':tn_ab,'FP':fp_ab,'FN':fn_ab,'Accuracy':acc_ab,'Sensitivity':sensitivity_ab,'Specificity':spec_ab})
knn = pd.Series({'TP':tp_knn,'TN':tn_knn,'FP':fp_knn,'FN':fn_knn,'Accuracy':acc_knn,'Sensitivity':sensitivity_knn,'Specificity':spec_knn})
df_summary = pd.DataFrame({"DecisionTree":dt,'RandomForest':rf,'AdaptiveBoosting':ab,"KNearestNeighbor": knn})
df_summary
df_summary.round(2)

,DecisionTree,RandomForest,AdaptiveBoosting,KNearestNeighbor
TP,775.00,561.00,585.00,434.00
TN,11124.00,11643.00,11585.00,11510.00
FP,861.00,342.00,400.00,475.00
FN,804.00,1018.00,1018.00,1145.00
Accuracy,0.88,0.90,0.90,0.88
Sensitivity,0.49,0.36,0.36,0.27
Specificity,0.93,0.97,0.97,0.96


## AdaBoosting Theory

In [65]:
##Weighted Sampling
pd.Series(np.random.randint(1,100,100)).value_counts().shape

(63,)

In [73]:
# random integers with given probabilities
random_samples=[]
for i in range(100):
    sample = np.random.choice([1,2,3],p=[0.1,0.3,0.6])
    random_samples.append(sample)
pd.Series(random_samples).value_counts()

3    60
2    33
1     7
dtype: int64

In [ ]:
# Classification Error Rate , Random columns and rows for decision tree 1

samples_dt1 = train.iloc[np.random.randint(1,train.shape[0],train.shape[0])]
all_cols = np.array(train.drop('Attrition',axis = 1).columns)
cols_position = np.random.randint(1,len(all_cols),3)
random_cols = all_cols[cols_position]

samples_dt1_x = samples_dt1[random_cols]
samples_dt1_y = samples_dt1['Attrition']
random_cols

dt1 = DecisionTreeClassifier(max_depth = 1)
dt1.fit(samples_dt1_x,samples_dt1_y)
test_pred = dt1.predict(test_x[random_cols])
df_pred = pd.DataFrame({'actual':test_y,'predicted':tesy_pred})
df_pred

In [77]:
import math
0.5 * math.log((1-0.158)/0.158)

0.8365924906076799